In [308]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas
import seaborn
from sklearn.metrics import confusion_matrix
%matplotlib inline
import time

In [309]:
#Load DATA

def loadMNIST( prefix, folder ):
    intType = np.dtype( 'int32' ).newbyteorder( '>' )
    nMetaDataBytes = 4 * intType.itemsize

    data = np.fromfile( folder + "/" + prefix + '-images-idx3-ubyte', dtype = 'ubyte' )
    magicBytes, nImages, width, height = np.frombuffer( data[:nMetaDataBytes].tobytes(), intType )
    data = data[nMetaDataBytes:].astype( dtype = 'float32' ).reshape( [ nImages, width, height ] )

    labels = np.fromfile( folder + "/" + prefix + '-labels-idx1-ubyte',
                          dtype = 'ubyte' )[2 * intType.itemsize:]

    return data, labels

train_x, train_y_cls = loadMNIST( "train", "../Simple Linear Model/MNIST_data" )
test_x, test_y_cls = loadMNIST( "t10k", "../Simple Linear Model/MNIST_data" )

In [310]:
#Convert labels to One-Hot Encoding

def toHotEncoding( classification ):
    # emulates the functionality of tf.keras.utils.to_categorical( y )
    hotEncoding = np.zeros( [ len( classification ), 
                              np.max( classification ) + 1 ] )
    hotEncoding[ np.arange( len( hotEncoding ) ), classification ] = 1
    return hotEncoding

train_y = toHotEncoding( train_y_cls )
test_y = toHotEncoding( test_y_cls )

In [311]:
#Data info
img_size = 28*28
num_classes = 10

train_x = train_x.reshape(-1,784)
test_x = test_x.reshape(-1,784)

In [312]:
#TensorFlow Graph

#PlaceHolder Variables
x = tf.placeholder(tf.float32, [None, img_size])
y_true = tf.placeholder(tf.float32, [None,num_classes])
y_true_cls = tf.placeholder(tf.int64, [None])

#Variables to be Optimized
weights = tf.Variable(tf.zeros([img_size,num_classes]))
biases = tf.Variable(tf.zeros([num_classes]))


In [313]:
#Model

logits = tf.matmul(x,weights) + biases
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis=1)
#Loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits,
                                                       labels = y_true)
#Cost
cost = tf.reduce_mean(cross_entropy)

In [314]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.3).minimize(cost)

#Performance_metric
correct_pred = tf.equal(y_pred_cls,y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [315]:
#TensorFlow Run

features_placeholder = tf.placeholder(tf.float32, [None, img_size])
labels_placeholder = tf.placeholder(train_y.dtype, [None,num_classes])
#labels_true_cls = tf.placeholder(tf.int64, [None])

session = tf.Session()
session.run(tf.global_variables_initializer())

#Function for optimization
batch_size = 128

def optimize(num_iterations):
    for i in range(num_iterations):
        feed_dict_train = {x: train_x,
                           y_true: train_y}
        session.run(optimizer, feed_dict= feed_dict_train)


In [316]:
#Feed_dict_test
feed_dict_test={x: test_x,
                y_true: test_y,
                y_true_cls: test_y_cls}
feed_dict_test

{<tf.Tensor 'Placeholder_61:0' shape=(?, 784) dtype=float32>: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 <tf.Tensor 'Placeholder_62:0' shape=(?, 10) dtype=float32>: array([[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 <tf.Tensor 'Placeholder_63:0' shape=(?,) dtype=int64>: array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)}

In [317]:
#Performance
def print_accuracy():
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))


In [318]:
#Accuracy without any optimisation
print_accuracy()

Accuracy on test-set: 9.8%


In [319]:
#Performance after 1 Optimisation
start_time = time.time()

optimize(1)

end_time = time.time()
print ("--- %s seconds ---" % (start_time-end_time))

In [320]:
print_accuracy()

Accuracy on test-set: 63.7%


In [321]:
#Performance after 100 Optimisation
start_time = time.time()

optimize(100)

end_time = time.time()
print ("--- %s seconds ---" % (start_time-end_time))

--- -16.891024112701416 seconds ---


In [322]:
print_accuracy()

Accuracy on test-set: 88.1%


In [323]:
#Performance after 1000 Optimisation (We have already performed 100 iterations)
start_time = time.time()

optimize(900)

end_time = time.time()
print ("--- %s seconds ---" % (start_time-end_time))

--- -167.42712211608887 seconds ---


In [324]:
print_accuracy()

Accuracy on test-set: 81.7%


In [325]:
session.close()